<a href="https://colab.research.google.com/github/fatemafaria142/Analyzing-Large-Language-Models-with-LangChain/blob/main/Agents_interaction_with_Single_CSV_format_using_Llama_2_7b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
!pip install langchain_experimental -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 kB 5.2 MB/s eta 0:00:00


### **Import Necessary Libraries**

In [3]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

### **Access Dataset**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd
# Read the CSV file
df = pd.read_csv("/content/drive/MyDrive/titanic.csv")

In [26]:
# Display the first few rows
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# Display the last few rows
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


In [9]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [10]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [11]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [12]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Using **`ZERO_SHOT_REACT_DESCRIPTION`**
##### **NOTE:** this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.
* **Documentation Link:** https://python.langchain.com/docs/integrations/toolkits/csv

* When the temperature is set to a value close to zero (but not zero), it makes the model's sampling almost deterministic, choosing the most probable token with very high likelihood. However, **when the temperature is exactly zero, it would effectively mean dividing by zero or multiplying by infinity, leading to invalid probabilities.**

* If you're looking to have greedy decoding (always choosing the most probable next token), you can set `do_sample=False `and not worry about the temperature. However, if you want to use the do_sample=True option and still want the generation to be almost deterministic, set the temperature to a very `small value like 0.001.`

In [13]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
temperature =0.001  # You can adjust this value based on your preference
pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=112, do_sample=do_sample, pad_token_id=pad_token_id, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

**List of Tools**
* **`python_repl -`** A Python shell. Use this to execute python commands. Input should be a valid python command. If you expect output it should be printed out.
* **`serpapi [Complete] -`** A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
* **`wolfram-alpha [Complete] - `**A wolfram alpha search engine. Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Input should be a search query.
* **`requests -`** A portal to the internet. Use this when you need to get specific content from a site. Input should be a specific url, and the output will be all the text on that page.
* **`terminal -`** Executes commands in a terminal. Input should be valid commands, and the output will be any output from running that command.
* **`pal-math -`** A language model that is excellent at solving complex word math problems. Input should be a fully worded hard word math problem.
* **`pal-colored-objects -`** A language model that is wonderful at reasoning about position and the color attributes of objects. Input should be a fully worded hard reasoning problem. Make sure to include all information about the objects AND the final question you want to answer.
* **`llm-math -`** Useful for when you need to answer questions about math.
* **`open-meteo-api -`** Useful for when you want to get weather information from the OpenMeteo API. The input should be a question in natural language that this API can answer.
* **`news-api -`** Use this when you want to get information about the top headlines of current news stories. The input should be a question in natural language that this API can answer.
* **`tmdb-api -`** Useful for when you want to get information from The Movie Database. The input should be a question in natural language that this API can answer.
* **`google-search -`** A wrapper around Google Search. Useful for when you need to answer questions about current events. Input should be a search query.

* To stop an AgentExecutor in LangChain after it has been dispatched or if it takes too long to respond, you can set the max_execution_time parameter when initializing the agent. For example, to stop the agent after 1 second, you can use `max_execution_time=1`

In [14]:
df = "/content/drive/MyDrive/titanic.csv"
agent = create_csv_agent(
    llm,
    df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_execution_time=1
)

In [15]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': '|    |   PassengerId |   Survived |   Pclass | Name                                                | Sex    |   Age |   SibSp |   Parch | Ticket           |    Fare | Cabin   | Embarked   |\n|---:|--------------:|-----------:|---------:|:----------------------------------------------------|:-------|------:|--------:|--------:|:-----------------|--------:|:--------|:-----------|\n|  0 |             1 |          0 |        3 | Braund, Mr. Owen Harris                             | male   |    22 |       1 |       0 | A/5 21171        |  7.25   | nan     | S          |\n|  1 |             2 |          1 |        1 | Cumings, Mrs. John Bradley (Florence Briggs Thayer) | female |    38 |       1 |       0 | PC 17599         | 71

#### **Exploring some examples using agent**

In [16]:
agent.run("how many rows are there?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: I should count the number of rows in the dataframe to determine the answer to the question.
Action: Use `len(df)` to count the number of rows in the dataframe.
Action Input: `len(df)`Use `len(df)` to count the number of rows in the dataframe. is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

In [17]:
agent.run("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: I can use the `sibsp` column to count the number of people with more than 3 siblings
Action: Use the `sibsp` column to count the number of people with more than 3 siblings
Action Input: `df['sibsp'].value_counts()`Use the `sibsp` column to count the number of people with more than 3 siblings is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

In [18]:
agent.run("whats the square root of the average age?")



> Entering new AgentExecutor chain...
Thought: I can calculate the average age by taking the mean of the ages in the dataframe.
Action: Use the `mean()` function to calculate the average age.
Action Input: `df['Age'].mean()`Use the `mean()` function to calculate the average age. is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

In [19]:
agent.run("How many passengers survived, and how many did not?")



> Entering new AgentExecutor chain...
Thought: I can use the `survived` column to count the number of passengers who survived.
Action: Use the `df.survived.value_counts()` function to count the number of passengers who survived.
Action Input: `df.survived.value_counts()`Use the `df.survived.value_counts()` function to count the number of passengers who survived. is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

In [20]:
agent.run("What is the average ticket fare for each passenger class?")



> Entering new AgentExecutor chain...
Thought: I can calculate the average ticket fare for each passenger class by summing the fare for each passenger and dividing by the total number of passengers in that class.
Action: Use the `df.groupby('Pclass').mean()` method to calculate the average ticket fare for each passenger class.
Action Input: `df.groupby('Pclass').mean()`Use the `df.groupby('Pclass').mean()` method to calculate the average ticket fare for each passenger class. is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

In [21]:
agent.run("How many passengers traveled with family members?")



> Entering new AgentExecutor chain...
Thought: Let's count the number of passengers who have 'SibSp' equal to 1.
Action: Use df['SibSp'] == 1 to count the number of passengers.
Action Input: df['SibSp'] == 1Use df['SibSp'] == 1 to count the number of passengers. is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

In [22]:
agent.run("How many passengers have information recorded for their cabin?")



> Entering new AgentExecutor chain...
Thought: Hmm, let me check...
Action: [python_repl_ast]
Action Input: `df.info()`[python_repl_ast] is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

In [23]:
agent.run("What is the average age of passengers who survived versus those who did not?")



> Entering new AgentExecutor chain...
Thought: I can use the `mean()` function to calculate the average age of the survivors and non-survivors.
Action: Use `mean()` to calculate the average age of the survivors and non-survivors.
Action Input: `df['Survived'].mean()`Use `mean()` to calculate the average age of the survivors and non-survivors. is not a valid tool, try one of [python_repl_ast].

> Finished chain.


'Agent stopped due to iteration limit or time limit.'